In [1]:
import os

In [2]:
!cd /content
!rm -rf /content/mt

In [3]:
# clone if does not exist
if not os.path.isdir('/content/mt'):
    !git clone https://github.com/fbelderink/mt.git
%cd /content/mt
!git pull
!git checkout fb/dev
!git pull

Cloning into 'mt'...
remote: Enumerating objects: 836, done.
remote: Counting objects: 100% (836/836), done.
remote: Compressing objects: 100% (405/405), done.
remote: Total 836 (delta 429), reused 795 (delta 397), pack-reused 0
Receiving objects: 100% (836/836), 3.42 MiB | 21.74 MiB/s, done.
Resolving deltas: 100% (429/429), done.
/content/mt
Already up to date.
Branch 'fb/dev' set up to track remote branch 'fb/dev' from 'origin'.
Switched to a new branch 'fb/dev'
Already up to date.


In [4]:
import training.train as train
from utils.ConfigLoader import ConfigLoader
from utils.hyperparameters import Hyperparameters
from google.colab import drive

In [21]:
drive_dir = '/content/drive'
drive.mount(drive_dir)

dataset_dir = os.path.join(drive_dir,'MyDrive/data/mt')

model_hyperparameters = Hyperparameters(
    ConfigLoader("configs/config.yaml").get_config())

train.train(os.path.join(dataset_dir,
                        f"train7k_w{model_hyperparameters.window_size}.pt"),
            os.path.join(dataset_dir,
                        f"val7k_w{model_hyperparameters.window_size}.pt"),
            model_hyperparameters,
            val_rate=400,
            train_eval_rate=200,
            num_workers=2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
training on cuda:0
Number of Batches: 2094
Batch Size: 200
total number of trainable parameters: 12108483
Layers:
source_embedding: Embedding(6797, 150)
target_embedding: Embedding(6461, 150)
fc_source: Linear(in_features=1050, out_features=600, bias=True)
fc_source_bn: BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
dropout_source: Dropout(p=0.05, inplace=False)
fc_target: Linear(in_features=450, out_features=600, bias=True)
fc_target_bn: BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
dropout_target: Dropout(p=0.05, inplace=False)
fc1: Linear(in_features=1200, out_features=1200, bias=True)
fc1_bn: BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
dropout1: Dropout(p=0.05, inplace=False)
fc2: Linear(in_features=1200, out_features=6461, bias=True)
fc2_bn: BatchNo

KeyboardInterrupt: 

In [9]:
from datetime import datetime
date = datetime.today().strftime('%d-%m-%Y')

In [23]:
import torch
from assignments.assignment4 import *
from utils.file_manipulation import load_data
from preprocessing.dictionary import Dictionary

eval_all = True

data_de = load_data("data/raw/multi30k.dev.de")
data_en = load_data("data/raw/multi30k.dev.en")

dict_de = Dictionary.load(os.path.join(dataset_dir, "dicts/train_dict_de.pkl"))
dict_en = Dictionary.load(os.path.join(dataset_dir, "dicts/train_dict_en.pkl"))

if eval_all:
  determine_models_bleu(f"eval/checkpoints/{date}", data_de, data_en,
                      dict_de, dict_en, 3,
                      model_hyperparameters.window_size, True)
else:
  model = torch.load(f"eval/checkpoints/{date}/15_01_58.pth")
  test_model_bleu(model, data_de, data_en, dict_de, dict_en,
                 3, model_hyperparameters.window_size, True, None)


determine bleu of eval/checkpoints/13-06-2024/15_01_58.pth
Model BLEU: 0.2742871839806815
determine bleu of eval/checkpoints/13-06-2024/15_01_19.pth
Model BLEU: 0.2777224360095151
determine bleu of eval/checkpoints/13-06-2024/15_00_05.pth
Model BLEU: 0.2504418786978578
determine bleu of eval/checkpoints/13-06-2024/15_00_42.pth
Model BLEU: 0.2716663025217683
determine bleu of eval/checkpoints/13-06-2024/14_59_27.pth
Model BLEU: 0.21416791624619436


In [24]:
!zip -r {dataset_dir}/checkpoints.zip eval/checkpoints/{date}/

  adding: eval/checkpoints/13-06-2024/ (stored 0%)
  adding: eval/checkpoints/13-06-2024/15_01_58.pth (deflated 7%)
  adding: eval/checkpoints/13-06-2024/15_01_19.pth (deflated 7%)
  adding: eval/checkpoints/13-06-2024/15_00_05.pth (deflated 7%)
  adding: eval/checkpoints/13-06-2024/15_00_42.pth (deflated 7%)
  adding: eval/checkpoints/13-06-2024/14_59_27.pth (deflated 8%)


In [20]:
!rm -rf eval/checkpoints/{date}